# Basic gym Environment and Data

In [78]:
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiDiscrete, MultiBinary, flatdim, flatten_space, unflatten, flatten

import numpy as np
import sys
import pandas as pd
import random
from collections import OrderedDict

In [67]:
class VPPBiddingEnv(Env):
    
    def __init__(self,
                 renewables_df, 
                 bids_df,
                 tenders_df,
                 hist_window_size,
                 forecast_window_size,
                 frame_bound
                ):
        
        # data 
        self.renewables_df = renewables_df
        self.tenders_df = tenders_df
        self.bids_df = bids_df
        self.total_FCR_demand = None
        
        # window_size
        self.hist_window_size = hist_window_size
        self.forecast_window_size = forecast_window_size
        self.frame_bound = frame_bound
        
        self.hydro_df, self.wind_df = self._process_data()
        
        # episode
        self.start_date = pd.to_datetime(self.frame_bound[0])
        self.end_date = pd.to_datetime(self.frame_bound[1])
        
        # as we need historic data the current date mus be the (given start) PLUS (historic data window size)  = (first day of the auction) 
        self.auction_date = self.start_date  +  pd.offsets.DateOffset(days= self.hist_window_size)
        
        self.initial = True
        self.done = None
        self.total_reward = 0.
        self.total_profit = 0.
        self.history = None
        
        # Slots 
        self.slots_won = [None, None, None, None, None, None]
        self.slot_prices = [None, None, None, None, None, None]
                
        # Spaces
        low = np.float32(np.array([0.0] * 96)) #96 timesteps to min 0.0
        high = np.float32(np.array([1.0] * 96)) #96 timesteps to max 1.0
        
        # Create a nested observation space with all observations inside
        self.nested_observation_space = Dict({
            'historic_data': Dict({
                "hydro_historic": Box(low, high, dtype=np.float32),
                "wind_historic":  Box(low, high, dtype=np.float32)
            }),
            'forecast_data':  Dict({
                "hydro_forecast": Box(low, high, dtype=np.float32),
                "wind_forecast": Box(low, high, dtype=np.float32),
                "soc_forecast": Box(low, high, dtype=np.float32)
            }),
            'market_data':  Dict({
                "market_demand": Discrete(3), # for the demands 573, 562 and 555 MW
                # TODO for 2021 its always 562, how to handle differetn years? maybe set it as a global constant? 
                # TODO: evtl delete market demand ? 
                "predicted_market_prices":  Box(low=0.0, high=np.float32(1634.52), shape=(6, 1), dtype=np.float32), # for each slot, can be prices of same day last week 
            }),
            'time_features':  Dict({
                "weekday": Discrete(7), # for the days of the week
                "holiday": Discrete(2), # holiday = 1, no holiday = 0
                "month": Discrete(12), # for the month
            }),
            "auction_results":  Dict({
                "slots_won": MultiBinary(5), #boolean for each slot, 0 if loss , 1 if won 
                "slot_prices": Box(low=0.0, high=np.float32(4257.07), shape=(6,), dtype=np.float32)
            })
        })
        

        self.action_space = Tuple((
            # INFO: TSOs allow divisible and indivisible bids. Biggest divisible bid was 188 MW , maximum price was 4257.07 
            MultiDiscrete([ 188, 188, 188, 188, 188 , 188]),
            Box(low=0.0, high=np.float32(4257.07), shape=(6,), dtype=np.float32)))
    
    
    def reset(self):
        
        if self.initial is False:
            self.auction_date =  self.auction_date + pd.offsets.DateOffset(days=1)
        
        # Auction time is officially 08.00 of the auction day (1 day before delivery day). 
        # The algorithm run time is at 07:00 shortly before the auction at 08:00 to use the latest forecasts. 
        self.auction_time =  self.auction_date + pd.offsets.DateOffset(hour=7)
        self.total_FCR_demand = self.tenders_df[str(self.auction_date):]["total"][0] 
        print('total_FCR_demand = %s ' % self.total_FCR_demand)
        self.done = False

        # reset for each episode 
        self._get_new_timestamps()
        
        # when first Episode is finished, set boolean.  
        self.initial = False
        
        return self._get_observation()
        
    
    def _get_new_timestamps(self):
        
        self.historic_data_start = self.auction_time - pd.offsets.DateOffset(days= self.hist_window_size)
        self.historic_data_end =  self.auction_time
        
        self.forecast_start = self.auction_date + pd.offsets.DateOffset(days=1) 
        self.forecast_end = self.auction_date + pd.offsets.DateOffset(days=self.forecast_window_size + 1) 
        
        self.market_start = self.auction_date + pd.offsets.DateOffset(days= 1)
        self.market_end = self.market_start + pd.offsets.DateOffset(hours=20)
        
        self.slot_date_list = []
        slot_date = self.market_start 
        for i in range(0,6):
            self.slot_date_list.append(str(slot_date))
            slot_date = slot_date + pd.offsets.DateOffset(hours=4)   
    
    
    def _get_observation(self): 
            
        hydro_historic = self.hydro_df[str(self.historic_data_start) : str(self.historic_data_end)].to_numpy()
        wind_historic = self.wind_df[str(self.historic_data_start) : str(self.historic_data_end)].to_numpy()
        
        hydro_forecast = self.hydro_df[str(self.historic_data_start) : str(self.historic_data_end)].to_numpy()
        wind_forecast =  self.wind_df[str(self.forecast_start) : str(self.forecast_end)].to_numpy()
        
        if self.total_FCR_demand == 573: 
            # in 2020
            market_demand = 0
        if self.total_FCR_demand == 562:
            # in 2021
            market_demand = 1
        if self.total_FCR_demand == 555:
            # in 2021
            market_demand = 2
        else: 
            # default
            market_demand = 1
        # TODO: validate if market demand can be induce in another way, maybe on function call ? 
        
        predicted_market_prices = [10. , 20. , 10. , 20. , 10. , 20.] # TODO: naive prediction: retrieve price of same day last week 
        weekday = random.randrange(7) # TODO: retrieve from time dataframe 
        holiday = random.randrange(2) # TODO: retrieve from time dataframe 
        month = random.randrange(12) # TODO: retrieve from time dataframe 
        
        if self.auction_date == (self.start_date + pd.offsets.DateOffset(days=1)):  
            self.slots_won = [0, 0, 0, 0, 0, 0]
            self.slot_prices = [0., 0., 0., 0., 0., 0.]
        
        next_observation = OrderedDict({
            "historic_data": OrderedDict({
                "hydro_historic": hydro_historic,
                "wind_historic": wind_historic
                }),
            "forecast_data": OrderedDict({
                "hydro_forecast": hydro_forecast,
                "wind_forecast": wind_forecast
            }),
            "market_data": OrderedDict({
                "market_demand": market_demand,
                "predicted_market_prices": predicted_market_prices
            }),
            "time_features": OrderedDict({
                "weekday": weekday, 
                "holiday": holiday, 
                "month": month
            }), 
            "auction_results": OrderedDict({
                "slots_won": self.slots_won,
                "slot_prices": self.slot_prices
            })
        })  
            
        return next_observation
    
    
    def step(self, action):
        # take the bid out of the action of the agent and resimulate the market clearing algorithm
        self._simulate_market(action)
        
        # calculate reward from state and action 
        step_reward = self._calculate_reward(action)
        self.total_reward += step_reward
        self._update_profit(action)
    
        info = dict(
            auction_time = str(self.auction_time),
            step_reward = step_reward,
            total_reward = self.total_reward,
            total_profit = self.total_profit
        )
        self._update_history(info)
        
        observation = self._get_observation()
        
        self.done = True
        
        return observation, step_reward, self.done, info
    
    
    def _calculate_reward(self, action):        
        # TODO: validate if capacity could be provided
        # TODO: write reward function
        
        step_reward = 1.0
        return step_reward
    
    
    def _update_profit(self, action):
        # for won slot calculate the profit by multiplicating bid size and bid prize 
        
        # slot profit 
        # day profit 
        
        trade = False 
        if trade == False: 
            self.total_profit +=0
        if trade == True: 
            self.total_profit = 1
        # TODO: implement profit function 

    
    def _update_history(self, info):
        if not self.history:
            self.history = {key: [] for key in info.keys()}

        for key, value in info.items():
            self.history[key].append(value)

            
    def render(self):
        # TODO: Implement visulisation
        pass
    
    
    def _process_data(self):
        hydro_df = self.renewables_df.loc[:, 'Hydro1']
        wind_df = self.renewables_df.loc[:, 'WP1']
        # TODO: add more power plants
        return hydro_df, wind_df
    

    def _simulate_market(self, action):
        
        auction_bids = self.bids_df[self.market_start : self.market_end]
        print("Auction Date and time (D-1) = %s" % (self.auction_time))
        print("Historic Data Window: from %s to %s " % (self.historic_data_start, self.historic_data_end))
        print("Forecast Data Window: from %s to %s " % (self.forecast_start, self.forecast_end))

        
        for slot in range(0, len(self.slot_date_list)):
            slot_date = self.slot_date_list[slot]
            print("Slot Date (D) = %s" % (slot_date)) 
            slot_bids = auction_bids[slot_date : slot_date].reset_index(drop=True).reset_index(drop=False)
            slot_bids_list = slot_bids.to_dict('records')
            # extract the bid size out of the agents action
            agents_bid_size = action[0][slot]
            # extract the bid price out of the agents action
            agents_bid_price = action[1][slot]
            print("agents_bid_size = %s" % (agents_bid_size))
            print("agents_bid_price = %s" % (agents_bid_price))            
            # get settlement price
            settlement_price_DE = [bid['settlement_price'] for bid in slot_bids_list if bid['country']== "DE"][0] 
            print( "settlement_price_DE : " + str(settlement_price_DE))
            # First check if agents bid price is higher than the settlement price of Germany 
            if agents_bid_price > settlement_price_DE:
                # if it is higher, the slot is lost. 
                self.slots_won[slot] = 0
                # set settlement price for the current auctioned slot in slot_prices list
                self.slot_prices[slot] = settlement_price_DE
            else: 
                # If agents bid price is lower than settlement price (bid could be in awarded bids)
                # get CBMP of countries without LMP
                unique_country_bids = list({v['country']:v for v in slot_bids_list}.values())
                grouped_prices = [x['settlement_price'] for x in unique_country_bids]
                cbmp = max(set(grouped_prices), key = grouped_prices.count)
                print( "cbmp : " + str(cbmp))
                # check if settlement_price_DE is same as CBMP (no limit constraints where hit)
                if cbmp == settlement_price_DE:
                    price_filter = cbmp
                    print("DE has CBMP")
                else: 
                    # if Germany has a price based on limit constraints
                    price_filter = settlement_price_DE
                    print("DE has LMP")
                
                # TODO: TAKE BID SIZE ALSO INTO ACCOUNT!! THE BIGGER THE BID SIZE, THE MORE BIDS WILL BE REPLACE
                
                # get penultimate price of bid list with the same CBMP, 
                # as the probability is high that the agents bid moved the last bid out of the list.
                # sort the bid list based on the price
                sorted_slot_bids_list_by_price = sorted(slot_bids_list, key=lambda x: x['price'])
                new_settlement_price_DE = [bid['price'] for bid in sorted_slot_bids_list_by_price if bid['settlement_price']== price_filter][-2]
                # set boolean for auction win
                self.slots_won[slot] = 1
                # set settlement price for the current auctioned slot in slot_prices list
                self.slot_prices[slot] = new_settlement_price_DE
                print("self.slots_won = ")
                print("\n".join("won: \t{}".format(k) for k in self.slots_won))
                print("self.slot_prices = ")
                print("\n".join("price: \t{}".format(k) for k in self.slot_prices))
                    
           

### Data Preparation

In [68]:
# Renewables 
renewables_df = pd.read_csv("data/clean/renewables.csv", sep = ";").set_index("time", drop = True)

# Tenders (Demand) 
tenders_df = pd.read_csv("data/clean/tenders_all.csv", sep = ";", index_col = 0).set_index("SLOT_START", drop = True)
tenders_df.index = pd.to_datetime(tenders_df.index)
tenders_df = tenders_df.rename(columns={'TOTAL_DEMAND_[MW]': "total"})
#TODO: outsourcing of data cleaning to data cleaning notebook 

# Bids (Offers)
bids_df = pd.read_csv("data/clean/bids_all.csv", sep = ";", index_col = 0).set_index("SLOT_START", drop = True)
bids_df.index = pd.to_datetime(bids_df.index)
bids_df["indivisible"] = bids_df['NOTE'].str.contains(r'INDIVISIBLE', na=False)
bids_df = bids_df.rename(columns={'OFFERED_CAPACITY_PRICE_[EUR/MW]': 'price', 'OFFERED_CAPACITY_[MW]': 'offered', "COUNTRY" : "country", "SETTLEMENTCAPACITY_PRICE_[EUR/MW]": "settlement_price"})
bids_df["offered"] = bids_df["offered"].astype(int)
bids_df = bids_df[["offered", "price", "country", "settlement_price",  "indivisible"]]

hist_window_size = 1 # in days
forecast_window_size = 1 # in days
start_index = "2021-01-01 00:00:00+00:00"
end_index = "2021-12-30 00:00:00+00:00"
frame_bound = (start_index, end_index)

### Initilize Environment

In [69]:
env = VPPBiddingEnv(renewables_df = renewables_df,
                    bids_df = bids_df,
                    tenders_df = tenders_df,
                    hist_window_size = hist_window_size,
                    forecast_window_size = forecast_window_size,
                    frame_bound = frame_bound
                   )

### Run Episodes

In [70]:
#episodes = 365
episodes =3
score = 0

for episode in range(1, episodes+1):
    print('Start of Episode:{} '.format(episode))
    observation = env.reset()
    
    # timestep defined as: 1 step = 1 day.
    for timestep in range(1):
        #env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        score+=reward
        if done:
            print('Episode:{} Score:{} Info:{}'.format(episode, score, info))
            break
env.close()

Start of Episode:1 
total_FCR_demand = 1409.0 
Auction Date and time (D-1) = 2021-01-02 07:00:00+00:00
Historic Data Window: from 2021-01-01 07:00:00+00:00 to 2021-01-02 07:00:00+00:00 
Forecast Data Window: from 2021-01-03 00:00:00+00:00 to 2021-01-04 00:00:00+00:00 
Slot Date (D) = 2021-01-03 00:00:00+00:00
agents_bid_size = 123
agents_bid_price = 1768.3701
settlement_price_DE : 22.0
Slot Date (D) = 2021-01-03 04:00:00+00:00
agents_bid_size = 34
agents_bid_price = 3775.7852
settlement_price_DE : 17.94
Slot Date (D) = 2021-01-03 08:00:00+00:00
agents_bid_size = 116
agents_bid_price = 1613.5844
settlement_price_DE : 16.0
Slot Date (D) = 2021-01-03 12:00:00+00:00
agents_bid_size = 10
agents_bid_price = 4028.0728
settlement_price_DE : 14.56
Slot Date (D) = 2021-01-03 16:00:00+00:00
agents_bid_size = 56
agents_bid_price = 3307.5137
settlement_price_DE : 14.16
Slot Date (D) = 2021-01-03 20:00:00+00:00
agents_bid_size = 36
agents_bid_price = 1839.8267
settlement_price_DE : 16.1
Episode:1 Sc

In [46]:
display(bids_df.price.max())
display(bids_df.offered.max())

4257.07

188

In [71]:
display(env.nested_observation_space.sample())

OrderedDict([('auction_results',
              OrderedDict([('slot_prices',
                            array([1549.727  , 2410.0576 , 1992.7681 ,  843.65454,  100.88932,
                                    445.69736], dtype=float32)),
                           ('slots_won',
                            array([1, 1, 1, 0, 1], dtype=int8))])),
             ('forecast_data',
              OrderedDict([('hydro_forecast',
                            array([0.7830042 , 0.0488689 , 0.45766285, 0.8644078 , 0.8833698 ,
                                   0.8282799 , 0.75413156, 0.91917413, 0.7538613 , 0.82552075,
                                   0.35483733, 0.39570832, 0.10077941, 0.71005243, 0.55444753,
                                   0.14589798, 0.12269536, 0.9211999 , 0.18579394, 0.7661016 ,
                                   0.20743176, 0.39077815, 0.34083453, 0.09132111, 0.12283935,
                                   0.47861624, 0.82205653, 0.1670391 , 0.3919527 , 0.1476828 ,
        

In [73]:
display(env.action_space.sample())

(array([ 81,   8, 125,  47,  40,  50]),
 array([3298.0935 ,  945.89417, 2217.592  , 2728.8018 , 1782.7295 ,
        1591.266  ], dtype=float32))

In [74]:
env.nested_observation_space

Dict(auction_results:Dict(slot_prices:Box(0.0, 4257.07, (6,), float32), slots_won:MultiBinary(5)), forecast_data:Dict(hydro_forecast:Box(0.0, 1.0, (96,), float32), soc_forecast:Box(0.0, 1.0, (96,), float32), wind_forecast:Box(0.0, 1.0, (96,), float32)), historic_data:Dict(hydro_historic:Box(0.0, 1.0, (96,), float32), wind_historic:Box(0.0, 1.0, (96,), float32)), market_data:Dict(market_demand:Discrete(3), predicted_market_prices:Box(0.0, 1634.52, (6, 1), float32)), time_features:Dict(holiday:Discrete(2), month:Discrete(12), weekday:Discrete(7)))

In [75]:
flatdim(env.nested_observation_space)

521

In [76]:
flatten_space(env.nested_observation_space)

Box(0.0, [4.25706982e+03 4.25706982e+03 4.25706982e+03 4.25706982e+03
 4.25706982e+03 4.25706982e+03 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00

In [79]:
# flatten(space, x)
# Flatten a data point from a space.
# This is useful when e.g. points from spaces must be passed to a neural network, which only understands flat arrays of floats.
# Accepts a space and a point from that space. Always returns a 1D array. 

flatten(env.nested_observation_space, env.nested_observation_space.sample())

array([3.49842505e+03, 5.76912231e+02, 4.12260400e+03, 1.47692285e+03,
       3.45364185e+03, 5.28973877e+02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.50837481e-01,
       9.98444378e-01, 7.48157918e-01, 4.79269624e-01, 2.55715787e-01,
       4.66810673e-01, 1.45427343e-02, 8.16011429e-01, 2.37623706e-01,
       5.31285644e-01, 5.77462852e-01, 5.56173444e-01, 8.38075876e-01,
       2.38739282e-01, 4.97869134e-01, 3.50518059e-03, 8.94005895e-01,
       5.13246953e-01, 5.13044834e-01, 6.01146877e-01, 6.39608860e-01,
       2.02867076e-01, 5.18913507e-01, 6.94822133e-01, 9.98206794e-01,
       4.33132082e-01, 5.31369090e-01, 6.32009327e-01, 8.68398845e-01,
       5.84634840e-02, 3.00501853e-01, 6.93879962e-01, 9.21295762e-01,
       1.59146771e-01, 5.77703938e-02, 8.83434176e-01, 5.23113906e-01,
       8.18626344e-01, 6.31422579e-01, 4.76973534e-01, 7.42688060e-01,
       6.84605062e-01, 8.76637399e-01, 7.09031582e-01, 6.24756753e-01,
      

In [90]:
# check if flattened data point is in space

flatten(env.nested_observation_space, env.nested_observation_space.sample())  in flatten_space(env.nested_observation_space)


True

In [82]:

flattened_datapoint = flatten(env.nested_observation_space, env.nested_observation_space.sample())
display(flattened_datapoint)

unflattened_datapoint = unflatten(env.nested_observation_space, flattened_datapoint)
display(unflattened_datapoint)


array([2.66877869e+02, 4.13251611e+03, 3.03264453e+03, 4.18716699e+03,
       3.32012817e+03, 3.85569153e+02, 1.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 8.43169510e-01,
       2.00364664e-01, 3.02096754e-01, 3.41007501e-01, 1.28095046e-01,
       8.25501919e-01, 6.36377215e-01, 5.56993246e-01, 9.75947142e-01,
       6.87859356e-01, 9.85255897e-01, 9.24709141e-01, 6.66097179e-02,
       9.72169116e-02, 9.05357301e-01, 3.71238619e-01, 5.66721261e-01,
       9.82481897e-01, 6.82372272e-01, 2.45623067e-01, 4.60373819e-01,
       3.45963478e-01, 8.12900662e-01, 6.38466001e-01, 5.32905936e-01,
       1.16239242e-01, 2.31337354e-01, 8.52132142e-01, 5.69982111e-01,
       9.58067596e-01, 7.41546005e-02, 2.36876592e-01, 9.64604974e-01,
       1.08199634e-01, 2.00127572e-01, 4.09555763e-01, 3.33109677e-01,
       7.92968333e-01, 3.91974479e-01, 4.80736881e-01, 4.93660808e-01,
       5.74123085e-01, 2.26044208e-01, 8.45243931e-01, 3.65827024e-01,
      

OrderedDict([('auction_results',
              OrderedDict([('slot_prices',
                            array([ 266.87787, 4132.516  , 3032.6445 , 4187.167  , 3320.1282 ,
                                    385.56915], dtype=float32)),
                           ('slots_won',
                            array([1, 1, 0, 1, 0], dtype=int8))])),
             ('forecast_data',
              OrderedDict([('hydro_forecast',
                            array([0.8431695 , 0.20036466, 0.30209675, 0.3410075 , 0.12809505,
                                   0.8255019 , 0.6363772 , 0.55699325, 0.97594714, 0.68785936,
                                   0.9852559 , 0.92470914, 0.06660972, 0.09721691, 0.9053573 ,
                                   0.37123862, 0.56672126, 0.9824819 , 0.6823723 , 0.24562307,
                                   0.46037382, 0.34596348, 0.81290066, 0.638466  , 0.53290594,
                                   0.11623924, 0.23133735, 0.85213214, 0.5699821 , 0.9580676 ,
        

In [ ]:
# ----------------------
# other way of representing observation

'''
next_observation = Dict({
    'historic_data': Dict({
        "hydro_historic": Box(low, high, dtype=np.float32)
        "wind_historic":  Box(low, high, dtype=np.float32)
    }),
    'forecast_data':  Dict({
        "hydro_forecast": Box(low, high, dtype=np.float32),
        "wind_forecast": Box(low, high, dtype=np.float32),
        "soc_forecast": Box(low, high, dtype=np.float32)
        # TODO should I keep the Battery state of charge? 
    }),
    'market_data':  Dict({
        "market_demand": Discrete(3), # for the demands 573, 562 and 555 MW
        # TODO for 2021 its always 562, how to handle differetn years? maybe set it as a global constant? 

        "predicted_market_prices":  Box(low=0.0, high=1634.52, shape=(6, 1), dtype=np.float32), # for each slot, can be prices of same day last week 
    }),
    'time_features':  Dict({
        "weekday": Discrete(7), # for the days of the week
        "holiday": Discrete(2), # holiday = 1, no holiday = 0
        "month": Discrete(12), # for the month
    })
})
'''


# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states_hydro_historic = env.nested_observation_space["historic_data"]["hydro_historic"].shape
states_wind_historic = env.nested_observation_space["historic_data"]["wind_historic"].shape

actions = env.action_space[0].shape

In [ ]:
display(states_hydro_historic)
display(states_wind_historic)
display(actions)

In [ ]:
def build_model(states, actions):
    model = Sequential()
    # flatten? 
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model 


In [ ]:
model = build_model(states, actions)


In [ ]:
model.summary()


# 3. Build Agent with Keras-RL


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)


# 4. Reloading Agent from Memory


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)


In [ ]:
del model
del dqn
del env

In [ ]:
from gym.envs.registration import register

register(
    id='vpp-v0',
    entry_point='gym_foo.envs:FooEnv',
)


In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')


In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)


# Archive


In [ ]:
    def _simulate_market(self, action):
        
        # market clearing algorithm:
        
        # for each slot 
        # get all bids
        # bids to dict
        # add bid from action 
        # bring in order by price 
        # accumulate capacities until demand is filled 
        # check if bid is in bid list 
            # if yes, set auciton_won = True and get SETTLEMENTCAPACITY_PRICE
            # if no, set auciton_won = False
        
        
        
        
        
        
        
        ################################################
        
        
        # TODO: Market clearing algorithmus neu schreiben, Angebote aller Länder (ausser DÄNEMARK ?? ) müssen berücksichtigt werden, um gesatm demand zu füllen , erst dann steht preis für slot fest. 
        
        
        
        # for each slot 
        # get all bids
        # bids to dict
        # add bid from action 
        # bring in order by price
        # accumulate capacities until demand is filled  FOR ALL COUNTRIES
            # indivisible flag needs to be included and checked: indivisible offer needs to be fully included
            # check for every country, if Core Portion and export limit are satisfied 
                # for each country 
                    # at least the core portion needs to be satisfied
                    # if capacity is less than total demand, the settlement price for an underfilled country is the price of the last accepted bid
                # at most the export limit needs to be satisfied (all bids for Country - demand of country)
        
        # set prices for all countries
        
        
        ################################################
        
        # The optimisation algorithm calculates the optimal combination of FCR bids to be awarded under consideration of core shares and the maximum exchangeable FCR volumes (export limits of a country) with the goal to reduce total procurement cost of the cooperation. 
        
        # 1. If no export limits or core share constraint are hit, one cross-border marginal price (CBMP) will be determined equalling the most expensive awarded bid in the overall cooperation. 
        # Exceptions from having one CBMP may occur once export limits or core share constraint of one or more countries of the cooperation are hit. In this case, an LMP will be determined based on the local awarded bids within a country.
        
        

        def add_bid_to_acceppted_bids(bid,
                                     accepted_bids,
                                     sorted_bids_list_by_price,
                                     accumulated_capacities,
                                     LMPi=False):
            country_prefix = bid["country"]
            # add the capacity of the bid to the accumulated capacity of a single country
            accumulated_capacities[country_prefix + "_capacity"]  += bid["offered"]

            # add the capacity of each bid to the accumulated capacity
            accumulated_capacities["total_capacity"] += bid["offered"]
            if LMPi: 
                # if bid is evaluated for an LMPi
                accumulated_capacities[country_prefix + "_LMPi"] = bid["price"]
            else:
                # if it is a normal bid, the bids price is the new CBMP 
                accumulated_capacities["CBMP"] = bid["price"]
            # add the bid to the accepted bids list
            bid["allocated"] = bid["offered"]
            accepted_bids.append(bid)
            #print('bid["index"] = %s ' % (bid["index"]))
            #print("agents_bid_index = %s" % (agents_bid_index))
            #print("accumulated_capacities["total_capacity"] = %s" % (accumulated_capacities["total_capacity"]))

            # remove bid from list to not iterate over it again when searching for limit constraint replacement bids
            sorted_bids_list_by_price.remove(bid)

            return accepted_bids, sorted_bids_list_by_price, accumulated_capacities
            
        
        auction_bids = self.bids_df[self.market_start : self.market_end]
        country_constraints = self.tenders_df[self.market_start : self.market_end]

        
        
        for slot in range(0, len(self.slot_date_list)):
            slot_date = self.slot_date_list[slot]
            print("slot_date = %s" % (slot_date))
            slot_bids = auction_bids[slot_date : slot_date].reset_index(drop=True).reset_index(drop=False)
            bids_list = slot_bids.to_dict('records')
            
            slot_constraints = country_constraints[slot_date : slot_date].reset_index(drop=True).reset_index(drop=False)
            slot_constraints = slot_constraints.to_dict('records')[0]            

            #print("bids_list = ")
            #print("\n".join(" \t{}".format(k) for k in bids_list))
            
            # get the lenght of the list ot create an index fo the agents bid that now will be added
            agents_bid_index = len(bids_list)
            
            
            # extract the bid size out of the agents action
            # agents_bid_size = action[0][slot]
            agents_bid_size = 10

            
            # extract the bid price out of the agents action
            #agents_bid_price = action[1][slot]
            agents_bid_price = 0
            
            
            print("agents_bid_size = %s" % (agents_bid_size))
            print("agents_bid_price = %s" % (agents_bid_price))
            print("agents_bid_index = %s" % (agents_bid_index))
            # add the selected bid from the agent to the list of all bids
            bids_list.append({'index': agents_bid_index, 'offered': agents_bid_size, 'price': agents_bid_price, "country": "DE", "indivisible": False})
            # sort the list based on the price to later accumulate all bids' capacity (but ordered on price)
            sorted_bids_list_by_price = sorted(bids_list, key=lambda x: x['price'])
            
            #print("sorted_bids_list_by_price = ")
            #print("\n".join(" \t{}".format(k) for k in sorted_bids_list_by_price))
            
            country_list = list(set([x['country'] for x in sorted_bids_list_by_price]))
            LMPi_list = []
            accepted_bids = []
            slot_finished = False
            
            # CBMP = cross-border marginal price
            # LMPi = Local Marginal Price of importing country
            accumulated_capacities = {
                 'total_capacity': 0,
                 'CBMP': 0,
                 'DE_capacity': 0,
                 'DE_export': 0,
                 'DE_core': 0,
                 'DE_LMPi': 0,
                 'BE_capacity': 0,
                 'BE_export': 0,
                 'BE_core': 0,
                 'BE_LMPi': 0,
                 'FR_capacity': 0,
                 'FR_export': 0,
                 'FR_core': 0,
                 'FR_LMPi': 0,
                 'NL_capacity': 0,
                 'NL_export': 0,
                 'NL_core': 0,
                 'NL_LMPi': 0,
                 'AT_capacity': 0,
                 'AT_export': 0,
                 'AT_core': 0,
                 'AT_LMPi': 0,
                 'CH_capacity': 0,
                 'CH_export': 0,
                 'CH_core': 0,
                 'CH_LMPi': 0,
                 'SI_capacity': 0,
                 'SI_export': 0,
                 'SI_core': 0,
                 'SI_LMPi': 0,
                 'DK_capacity': 0,
                 'DK_export': 0,
                 'DK_core': 0,
                 'DK_LMPi': 0,
            }

            for bid in sorted_bids_list_by_price[:]:
                # check if LMPi_list containts countries that need to be checked
                if LMPi_list:
                    # check if current bid is from country
                    if bid["country"] != LMPi_list[0]:
                        # if not, go to next bid 
                        continue
                    # if bid is from country
                    else:
                        # add bid to accepted bids
                        accepted_bids, sorted_bids_list_by_price, accumulated_capacities = add_bid_to_acceppted_bids(bid,
                                                                                                                     accepted_bids,
                                                                                                                     sorted_bids_list_by_price,
                                                                                                                     accumulated_capacities,
                                                                                                                     LMPi = True
                                                                                                                    )
                        # after bid was added, remove country from LMPi list 
                        LMPi_list.pop(0)
                        
                        
                        
                # add bid to accepted bids
                accepted_bids, sorted_bids_list_by_price, accumulated_capacities = add_bid_to_acceppted_bids(bid,
                                                                                                            accepted_bids,
                                                                                                            sorted_bids_list_by_price,
                                                                                                            accumulated_capacities,
                                                                                                            LMPi = False)
                    
                # 2.1 Case of hitting a limit constraint
                # It is important to understand that an export limit or core share constraint is hit whenever it influences the solution and not only when the quantity awarded in a country is exactly equal to the respective limit quantity of that country.
                
            
                if accumulated_capacities["total_capacity"] >= self.total_FCR_demand:
                    # if accumulated_capacities["total_capacity"] is bigger than the demand, the last indvisible offer(s) need to be reduced
                    
                    # 2.1.2 Check if core share of every country is hit
                    for country in country_list: 
                        print("accumulated_capacities for " + country + ": " +  str(accumulated_capacities[country + "_capacity"]))
                        print("core constraint for " + country + ": " +  str(slot_constraints[country + "_core"]))
                        if accumulated_capacities[country + "_capacity"] < slot_constraints[country + "_core"]: 
                            print("CORE SHARE TOO SMALL FOR COUNTRY: " + country)
                            
                            del accepted_bids[-1]
                            accumulated_capacities["total_capacity"] -= bid["offered"]
                            LMPi_list.append(country)
                            
                            # TODO: set CBMP for all other countries. 
                            
                            # continue step continues for loop 
                            continue
                            
                            # TODO: for every core-underfilled country the capacity needs to be filled and the LMPi has to be found 
                        
                    # TODO: set allocated price of all bids :   bid["allocated"
                    
                    # TODO: Check Over Procurement 
                    if accumulated_capacities["total_capacity"] > self.total_FCR_demand: 
                        # get the overfilled capacity (difference)
                        overfilled_capacity = accumulated_capacities["total_capacity"] - self.total_FCR_demand
                        # get list of accepted bids that are divisible (= that is not indivisible)
                        accepted_bids_divisible = [bid for bid in accepted_bids if not bid['indivisible']]
                        # get last accepted bid that is divisible
                        accepted_bids_divisible[-1]
                        
                        # TODO: proceed with over procurement 
                        
                        # TODO: place slot_finished boolean somewhere
                        slot_finished = True
                    
                    # TODO: check if a country has a CBMP or LMPi 
                    
                    # last accepted bid sets settlement price of auction
                    settlement_price = bid["price"]
                    # set settlement price for the current auctioned slot in slot_prices list
                    self.slot_prices[slot] = settlement_price
                    
                    if agents_bid_index in [x['index'] for x in accepted_bids]:
                        # set boolean for auction win
                        self.slots_won[slot] = 1
                    
                    print("accumulated_capacities['total_capacity'] = %s" % (accumulated_capacities["total_capacity"]))
                    print("self.slots_won = ")
                    print("\n".join("won: \t{}".format(k) for k in self.slots_won))
                    print("self.slot_prices = ")
                    print("\n".join("price: \t{}".format(k) for k in self.slot_prices))
                    
                    
                    
                if slot_finished: 
                    break
                    

In [ ]:
 '''
                # add the selected bid from the agent to the list of all bids
                bids_list.append({'index': agents_bid_index, 'offered': agents_bid_size, 'price': agents_bid_price, "country": "DE", "indivisible": False})
                # sort the list based on the price to later accumulate all bids' capacity (but ordered on price)
                sorted_bids_list_by_price = sorted(bids_list, key=lambda x: x['price'])


                # as we dont habe enought data to simulate a realistic market clearing algorithm,
                # we calcualte a new settlement price with
                #prices = [x['price'] for x in accepted_bids]
                #diffs = np.diff(np.array(list_a))
                #mean_diff = sum(diffs) / len(diffs)
                '''

In [ ]:
demand = [{'index': 0, 'total': 1409.0, 'DE_demand': 562.0, 'DE_export': 168.0, 'DE_core': 169.0, 'BE_demand': 87.0, 'BE_export': 100.0, 'BE_core': 27.0, 'FR_demand': 508.0, 'FR_export': 152.0, 'FR_core': 153.0, 'NL_demand': 114.0, 'NL_export': 100.0, 'NL_core': 35.0, 'AT_demand': 71.0, 'AT_export': 100.0, 'AT_core': 22.0, 'CH_demand': 67.0, 'CH_export': 100.0, 'CH_core': 21.0, 'SI_demand': "nan", 'SI_export': "nan", 'SI_core': "nan", 'DK_demand': "nan", 'DK_export': "nan", 'DK_core': "nan"}]
demand = demand[0]
demand["DE_demand"]

In [ ]:
demand

In [ ]:
accepted_bids = [{'index': 0, 'offered': 11, "allocated" : 10, 'price': 0.0, 'country': 'DE', 'indivisible': False}
,{'index': 1, 'offered': 1, "allocated" : 10,'price': 0.0, 'country': 'DE', 'indivisible': True}
,{'index': 2, 'offered': 1,"allocated" : 10, 'price': 0.0, 'country': 'FR', 'indivisible': True}
,{'index': 3, 'offered': 4,"allocated" : 10, 'price': 0.0, 'country': 'DE', 'indivisible': False}]

country_tenders = [{"DE": 
                    {"total": 0, "export": 11, "core" : 10},
                    "BE": 
                    {"total": 0, "export": 11, "core" : 10}}]

country_capacity =  {"DE": 555, "BE": 200}        

# go through all accepted bids in reversed order sorted by price
for bid in reversed(accepted_bids):
    # if an order is divisible and can be divided...
    if not bid["indivisible"]:
        # check if the offered capacity of the bid is a minimum of 1 bigger than the overfilled_capacity (so it can be substracted)
        if bid["offered"] > overfilled_capacity: 
            difference_to_core = country_tenders[bid["country"]]["core"]
            overfilled_capacity
        bid["allocated"] = bid["allocated"]-1
        
        break

print(accepted_bids)


In [ ]:
accepted_bids = [{'index': 0, 'offered': 11, "allocated" : 10, 'price': 1.0, 'country': 'DE', 'indivisible': False}
,{'index': 1, 'offered': 1, "allocated" : 10,'price': 1.0, 'country': 'BE', 'indivisible': True}
,{'index': 2, 'offered': 1,"allocated" : 10, 'price': 2.0, 'country': 'FR', 'indivisible': True}
,{'index': 3, 'offered': 4,"allocated" : 10, 'price': 1.0, 'country': 'DE', 'indivisible': False}]


In [ ]:
accepted_bids

In [ ]:
display([bid['price'] for bid in accepted_bids if bid['country']== "DE"])
display([bid['price'] for bid in accepted_bids if bid['country']== "DE"][-1])


In [ ]:
set(([x['country'] for x in accepted_bids]))
(([x['country'] for x in accepted_bids]))
(([x['price'] for x in accepted_bids]))

In [ ]:
unique_country_bids = list({v['country']:v for v in accepted_bids}.values())
display(unique_country_bids)
all_prices = [x['price'] for x in unique_country_bids]
display(all_prices)

cbmp = max(set(all_prices), key = all_prices.count)
display(cbmp)


In [ ]:
unique_country_bids = [{'index': 665, 'offered': 1, 'price': 11.0, 'country': 'DE', 'settlement_price': 13.3, 'indivisible': False}, {'index': 598, 'offered': 1, 'price': 4.48, 'country': 'FR', 'settlement_price': 4.48, 'indivisible': False}, {'index': 677, 'offered': 23, 'price': 217.76, 'country': 'BE', 'settlement_price': 217.76, 'indivisible': True}, {'index': 671, 'offered': 1, 'price': 12.5, 'country': 'NL', 'settlement_price': 13.3, 'indivisible': False}, {'index': 673, 'offered': 5, 'price': 13.3, 'country': 'CH', 'settlement_price': 13.3, 'indivisible': False}, {'index': 670, 'offered': 2, 'price': 12.4, 'country': 'AT', 'settlement_price': 13.3, 'indivisible': False}, {'index': 675, 'offered': 1, 'price': 50.0, 'country': 'DK', 'settlement_price': 50.0, 'indivisible': True}]

In [ ]:
display(unique_country_bids)
all_prices = [d['settlement_price'] for d in unique_country_bids]
display(all_prices)

cbmp = max(set(all_prices), key = all_prices.count)
display(cbmp)


In [ ]:
[bid['settlement_price'] for bid in accepted_bids if bid['country']== "DE"][0]
